In [1]:
import os
for dirname, _, filenames in os.walk('/home/megatonrex/ICGPRO/dir/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/megatonrex/ICGPRO/dir/input/archive/captions.txt
/home/megatonrex/ICGPRO/dir/input/archive/Images/3524436870_7670df68e8.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/500678178_26ce0f4417.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/1022454332_6af2c1449a.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/2535746605_8124bf4e4f.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/2633722629_5eeb649c09.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/2975018306_0e8da316f5.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/2877637572_641cd29901.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/3286222970_1fa445e38f.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/3599392711_8264881de2.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/2896668718_0c3cff910f.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/1670592963_39731a3dac.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Images/3565021218_d2bc1aa644.jpg
/home/megatonrex/ICGPRO/dir/input/archive/Imag

In [2]:
import os
import numpy as np
import pickle
from tqdm.notebook import tqdm
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

2025-02-09 10:20:25.274252: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 10:20:25.386232: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 10:20:25.513496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739096425.661448     802 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739096425.691524     802 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 10:20:25.944714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
#storing the dataset in the base directory.
BASE_DIR = '/home/megatonrex/ICGPRO/dir/input/archive'
#storing the processed files like pickle file in the working directory.
WORKING_DIR  = '/home/megatonrex/ICGPRO/dir/working'

In [4]:
model = VGG16()

#removing the last prediction layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
print(model.summary())

2025-01-22 05:54:26.698376: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-01-22 05:54:26.929236: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-22 05:54:27.148647: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-22 05:54:27.378893: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-22 05:54:35.998300: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
#extracting features from the image
features = {}
directory = os.path.join(BASE_DIR, 'Images')

for img_name in tqdm(os.listdir(directory)):
    # img_path = directory + '/' + img_name
    img_path = os.path.join(directory, img_name)
    #resizing the image.
    image = load_img(img_path, target_size= (224,224))
    #transforimg the image in to numbers.
    image = img_to_array(image)
    # image = image.reshape((1, image.shape[0], image.shape[1], image.shape(2)))
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    feature = model.predict(image, verbose = 0)
    image_id = img_name.split('.')[0]
    features[image_id] = feature

  0%|          | 0/8091 [00:00<?, ?it/s]

In [7]:
# storing the data
pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb'))

In [4]:
# loading the data
with open(os.path.join(WORKING_DIR, 'features.pkl'), 'rb') as f:
    features = pickle.load(f)

In [5]:
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [6]:
mapping = {}
for line in tqdm(captions_doc.split('\n')):
    #it will split the image and its corresponding caption.
    tokens = line.split(',')
    #here we are skipping those captions which have length less than 2.
    if len(line) < 2:
        continue
    img_id, caption = tokens[0], tokens[1]
    img_id = img_id.split('.')[0]
    #formatting the characters by joining them in a single string.
    caption = "".join(caption)
    #making an img a key of a list if not present in the mapping dictionary.
    if img_id not in mapping:
        mapping[img_id] = []
    mapping[img_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [7]:
len(mapping)

8091

In [8]:
def cleaning(mapping):
    for key, captions in mapping.items():
        #iterates the number of captions in each list times.
        for i in range(len(captions)):
            #taking one caption at a time.
            caption = captions[i]
            caption = caption.lower()
            caption = caption.replace('[^A-Za-z]', '')
            caption = caption.replace('\s+', ' ')
            #adding start and end tag in each caption and removing single digit words.
            caption = '<start> '+ " ".join([word for word in caption.split() if len(word)>1]) + ' <end>'
            captions[i] = caption

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_802/1448847460.py:9: SyntaxWarning: invalid escape sequence '\s'
  caption = caption.replace('\s+', ' ')


In [9]:
mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [10]:
cleaning(mapping)

In [11]:
mapping['1000268201_693b08cb0e']

['<start> child in pink dress is climbing up set of stairs in an entry way <end>',
 '<start> girl going into wooden building <end>',
 '<start> little girl climbing into wooden playhouse <end>',
 '<start> little girl climbing the stairs to her playhouse <end>',
 '<start> little girl in pink dress going into wooden cabin <end>']

In [12]:
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [13]:
len(all_captions)

40455

In [14]:
all_captions[:10]

['<start> child in pink dress is climbing up set of stairs in an entry way <end>',
 '<start> girl going into wooden building <end>',
 '<start> little girl climbing into wooden playhouse <end>',
 '<start> little girl climbing the stairs to her playhouse <end>',
 '<start> little girl in pink dress going into wooden cabin <end>',
 '<start> black dog and spotted dog are fighting <end>',
 '<start> black dog and tri-colored dog playing with each other on the road <end>',
 '<start> black dog and white dog with brown spots are staring at each other in the street <end>',
 '<start> two dogs of different breeds looking at each other on the road <end>',
 '<start> two dogs on pavement moving toward each other <end>']

In [15]:
tokenizer = Tokenizer()
#looking at all the captions and remembering every single word in it. Alse giving each word a unique number, like a tag.
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
print(word_index)

{'end': 1, 'start': 2, 'in': 3, 'the': 4, 'on': 5, 'is': 6, 'and': 7, 'dog': 8, 'with': 9, 'man': 10, 'of': 11, 'two': 12, 'white': 13, 'black': 14, 'boy': 15, 'are': 16, 'woman': 17, 'girl': 18, 'to': 19, 'wearing': 20, 'people': 21, 'at': 22, 'water': 23, 'young': 24, 'red': 25, 'brown': 26, 'an': 27, 'his': 28, 'blue': 29, 'dogs': 30, 'running': 31, 'through': 32, 'playing': 33, 'shirt': 34, 'while': 35, 'down': 36, 'little': 37, 'standing': 38, 'ball': 39, 'grass': 40, 'person': 41, 'child': 42, 'snow': 43, 'jumping': 44, 'over': 45, 'three': 46, 'front': 47, 'sitting': 48, 'small': 49, 'up': 50, 'a': 51, 'field': 52, 'holding': 53, 'large': 54, 'group': 55, 'by': 56, 'green': 57, 'yellow': 58, 'children': 59, 'walking': 60, 'men': 61, 'her': 62, 'into': 63, 'beach': 64, 'air': 65, 'near': 66, 'jumps': 67, 'mouth': 68, 'street': 69, 'runs': 70, 'for': 71, 'another': 72, 'riding': 73, 'from': 74, 'its': 75, 'as': 76, 'bike': 77, 'stands': 78, 'girls': 79, 'one': 80, 'outside': 81, '

In [16]:
maxLen = max(len(caption.split()) for caption in all_captions)
maxLen

31

In [17]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [ ]:
#generating a generator to get data in batch(this prevents session from crashing)
def dataGenerator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            captions = mapping[key]
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
